In [2]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

import matplotlib
%matplotlib inline

matplotlib.rcParams['font.size'] = 20
matplotlib.rcParams['figure.figsize'] = (14,9)
matplotlib.rcParams['savefig.bbox'] = 'tight'

In [65]:
import importlib
importlib.reload(resnet)

<module 'resnet' from '/Users/albertxu/AI4AM/models/resnet.py'>

In [6]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
import pickle
import resnet

import taichiUtil

In [117]:
def preprocess(ex):
    ex['image'] = tf.image.resize(tf.image.convert_image_dtype(ex['image'], tf.float32), [250, 250])
    return ex

def pred(ex):
    return (ex['n_seq'] % 10 == 0)


In [120]:
train = tfds.load('taichiSim', split='train').map(preprocess).filter(flter).batch(10)

In [126]:
train.as_numpy_iterator()

In [160]:
taichiUtil.dat2vec(z)

array([[-0.92720532,  0.19170223, -0.35352325, -0.63758558,  0.27806041,
        -0.70423704, -0.9171735 , -0.96169937,  0.        ,  0.96170008,
         0.09230772,  0.        ,  0.62019706],
       [-0.7789709 ,  0.48633233,  0.66862869, -0.08112147, -0.7618475 ,
         0.60673964, -0.72283632,  0.48158008,  0.        , -0.48157972,
        -0.48222539,  0.        , -0.0381093 ],
       [ 0.26570737,  0.76882315,  0.71354294, -0.20474648,  0.13693374,
         0.7925958 ,  0.87967128,  0.08187816,  0.        , -0.08187791,
        -0.1724806 ,  0.        , -0.21241236],
       [ 0.74408114,  0.9485274 , -0.52203274,  0.4269807 ,  0.69549406,
        -0.12622169, -0.14873271,  0.09647024,  0.        , -0.09647012,
         0.93993342,  0.        ,  0.67713666],
       [ 0.49710172,  0.65732813,  0.02225685, -0.83988363,  0.02866478,
        -0.63391912, -0.05488397,  0.93139273,  0.        , -0.93139213,
        -0.33360454,  0.        ,  0.52064562],
       [ 0.12530425,  0.608551

In [116]:
z['n_seq'] % 10 == 0

<tf.Tensor: shape=(20,), dtype=bool, numpy=
array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False])>

In [107]:
for z in train:
    break

In [109]:
z['ex_scale']

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0.21091919, 0.23315436, 0.3898561 , 0.46161216, 0.42456526,
       0.36879563, 0.4296146 , 0.3775429 , 0.21344475, 0.38494688,
       0.4292607 , 0.25382555, 0.34540907, 0.30966866, 0.23245691,
       0.453043  , 0.48114288, 0.3796794 , 0.24854463, 0.45958525],
      dtype=float32)>

In [104]:
taichiUtil.dat2vec(z)

TypeError: Scalar tensor has no `len()`

In [14]:
batch_size = 10
vals_ds = tfds.load('taichiSim', split=[
    tfds.core.ReadInstruction('tmp', from_=k, to=k+10, unit='%')
    for k in range(0, 100, 10)], batch_size=batch_size)
trains_ds = tfds.load('taichiSim', split=[
    (tfds.core.ReadInstruction('tmp', to=k, unit='%') +
     tfds.core.ReadInstruction('tmp', from_=k+10, unit='%'))
    for k in range(0, 100, 10)], batch_size=batch_size)

In [153]:
class MyMo(tf.keras.Model):
    def __init__(self, im_head, n_class=4):
        super(MyMo, self).__init__()
        
        self.im_head = im_head
        self.fc1 = tf.keras.layers.Dense(units=n_class, activation=tf.keras.activations.softmax)
#         self.fc2 = tf.keras.layers.Dense(units=n_class, activation=tf.keras.activations.softmax)

#     def call(self, inputs, training=None):
    def call(self, im, uvec, training=None):
#         im, uvec = inputs
        x = self.im_head(im, training=training)
        x = tf.concat([x, uvec], axis=-1)
        out = self.fc1(x, training=training)
        return out
        

In [154]:
resn = resnet.resnet_18(drop_p=0, n_class=2048)
model = MyMo(resn, n_class=2)
objective = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.0001)

In [82]:
def loss(m, x, y, _train=True):
    _y = m(x, training=_train)
    return objective(y_true=y, y_pred=_y)

def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets, _train=True)
        return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [83]:
train_losses = []
train_accs = []
test_losses = []
test_accs = []

In [155]:
for x in train:
    break
model(x['image'], taichiUtil.dat2vec(x))

<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
array([[0.46993318, 0.5300668 ],
       [0.49032217, 0.5096779 ],
       [0.4872703 , 0.51272964],
       [0.5056    , 0.49440008],
       [0.48715752, 0.5128424 ],
       [0.49752817, 0.50247186],
       [0.5255966 , 0.47440338],
       [0.50974506, 0.49025497],
       [0.510732  , 0.489268  ],
       [0.5020897 , 0.49791026]], dtype=float32)>

In [147]:
a = model.im_head(x['image'])
b = taichiUtil.dat2vec(x)

In [152]:
tf.concat([a, b], axis=-1)

<tf.Tensor: shape=(10, 2061), dtype=float32, numpy=
array([[ 5.2245375e-04,  4.8988278e-04,  5.0078891e-04, ...,
         5.8630836e-01,  0.0000000e+00, -3.3652925e-01],
       [ 5.2293635e-04,  4.8946624e-04,  5.0087686e-04, ...,
        -2.0492473e-01,  0.0000000e+00,  1.5815520e-01],
       [ 5.2124937e-04,  4.8952823e-04,  5.0065818e-04, ...,
        -6.2110686e-01,  0.0000000e+00,  9.4827080e-01],
       ...,
       [ 5.2165560e-04,  4.8970233e-04,  5.0077878e-04, ...,
        -5.8419704e-01,  0.0000000e+00, -6.0668898e-01],
       [ 5.2148494e-04,  4.8914819e-04,  5.0076074e-04, ...,
        -3.6132336e-02,  0.0000000e+00,  8.0280352e-01],
       [ 5.2131800e-04,  4.8948126e-04,  5.0075003e-04, ...,
         3.4572017e-01,  0.0000000e+00,  6.1257815e-01]], dtype=float32)>

In [143]:
model(*inp)

ValueError: Shape (10, 13) must have rank 0

In [84]:
train = trains_ds[0]
val = vals_ds[0]

In [ ]:
# Training loop
n_epoch = 500
for epoch in range(n_epoch):
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    test_epoch_loss_avg = tf.keras.metrics.Mean()
    test_epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    
    train2 = train.shuffle(buffer_size=50)
    for ex in train2:
        x = tf.image.convert_image_dtype(ex['image'], tf.float32)
        ys = taichiUtil.dat2vec(ex)
        # class_encode = tf.reshape(tf.one_hot((ys[:,3:7] > 0).astype(int), 2), (-1, 8))
        y = (ys[:,3] > 0).astype(int)
        l, grads = grad(model, x, y)

        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        epoch_loss_avg.update_state(l)
        epoch_accuracy.update_state(y, model(x, training=True))
    
    print(f'Epoch {epoch:03d}, Train Loss: {epoch_loss_avg.result():.3f}, Acc: {epoch_accuracy.result():.3f}')


13